In [1]:
# Import required modules
import cv2
import math
import time
import torch

In [3]:
# getting face using opencv
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    
    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    
    for i in range(detections.shape[2]):
        confidence = detections[0 , 0 , i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

In [4]:
# opencv face detector
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

# age deployment 
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

# gender deployment
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

In [5]:
# age list and gendre list
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

In [6]:
# Load network
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)

In [7]:
# device setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
# model setting accordingly to device
if device == 'cpu':
    ageNet.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    
    genderNet.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    
    faceNet.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
else:
    ageNet.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    ageNet.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    genderNet.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    genderNet.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    genderNet.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    genderNet.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

Using CPU device


In [13]:
# Open a video file or an image file or a camera stream
cap = cv2.VideoCapture(0)  #replace 0 with the path of video file for age and gender prediciton
padding = 20
while True:
    # Read frame

    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv2.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()
print("time : {:.3f}".format(time.time() - t))


In [12]:
# for images
frame = cv2.imread('sample1.jpg')
padding = 20
frameFace, bboxes = getFaceBox(faceNet, frame)
if not bboxes:
    print("No face Detected, Checking next frame")

for bbox in bboxes:
    # print(bbox)
    face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPreds = genderNet.forward()
    gender = genderList[genderPreds[0].argmax()]
    # print("Gender Output : {}".format(genderPreds))
    print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

    ageNet.setInput(blob)
    agePreds = ageNet.forward()
    age = ageList[agePreds[0].argmax()]
    print("Age Output : {}".format(agePreds))
    print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

    label = "{},{}".format(gender, age)
    cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
cv2.imshow("Age Gender From Image Demo", frameFace)
cv2.waitKey(0)
cv2.destroyAllWindows()

Gender : Male, conf = 1.000
Age Output : [[2.2225309e-05 5.8275257e-04 9.8269939e-01 5.2425021e-05 1.6299812e-02
  9.7018215e-05 2.1680613e-04 2.9518760e-05]]
Age : (8-12), conf = 0.983
Gender : Male, conf = 1.000
Age Output : [[2.2122951e-01 7.0148927e-01 7.7020638e-02 5.0334183e-05 6.6446075e-05
  8.4754574e-05 2.6727008e-05 3.2206233e-05]]
Age : (4-6), conf = 0.701
Gender : Female, conf = 0.992
Age Output : [[4.8369761e-06 5.5865039e-06 4.2791725e-03 6.6573936e-03 9.8674357e-01
  2.2955353e-03 9.4001989e-06 4.4530993e-06]]
Age : (25-32), conf = 0.987
